# 3View stack 10073

import statements

In [ ]:
# import modules
import glob
import os
import path
import ntpath
import stat
import sys
import copy

import numpy as np
import pandas as pd

# Image processing tools
import mrcfile

import skimage
import skimage.io
from imageio import imread, volread, imwrite, volwrite
from ipywidgets import fixed, interactive


#import caliban_toolbox.pre_annotation.data_loader
#from caliban_toolbox import reshape_data
#from caliban_toolbox.figure_eight_functions import create_figure_eight_job, download_figure_eight_output
#from caliban_toolbox.utils import widget_utils, plot_utils, data_utils, io_utils

from bokeh.palettes import Viridis, Greys
from bokeh.io import export_png

import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from skimage import filters, img_as_uint
perm_mod = stat.S_IRWXO | stat.S_IRWXU | stat.S_IRWXG

import bebi103

import bokeh
bokeh.io.output_notebook()

import panel as pn
pn.extension()

import colorcet

import holoviews as hv
hv.extension('bokeh')

In [ ]:
# look at the header of the bin4 data
mrc = mrcfile.open('/Volumes/LaCie/11-10073_SBEM2_bin10.mrc')
print(mrc.voxel_size)
print(mrc.header)
mrc.close

## IMOD trim vol commands
The IMOD trim vol commands used to extract the aggregate sub-volumes from the bin0 data (11-10073)

`#agg 4
Center (7679,9568); offset -845,-3904
Image size: 3623 x 3461;   To excise:
  trimvol -x 5868,9490 -y 7838,11298 -z 220,470 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg4.mrc
#agg 7 
Center (10468,5023); offset 1944,-8449
Image size: 2012 x 1936;   To excise:
  trimvol -x 9462,11473 -y 4055,5990 -z 60,190 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg7.mrc
#agg 5
Center (11819,9298); offset 3295,-4174
Image size: 3323 x 3281;   To excise:
  trimvol -x 10158,13480 -y 7658,10938 -z 240,450 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg5.mrc
#agg 3
Center (4744,6113); offset -3780,-7359
Image size: 3869 x 4049;   To excise:
  trimvol -x 2810,6678 -y 4089,8137 -z 260,504 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg3.mrc
#agg 2
Center (4557,12224); offset -3967,-1248
Image size: 7310 x 7860;   To excise:
  trimvol -x 902,8211 -y 8294,16153 -z 200,504 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg2.mrc
#agg 6
Center (12222,13069); offset 3698,-403
Image size: 3521 x 3131;   To excise:
  trimvol -x 10462,13982 -y 11504,14634 -z 220,460 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg6.mrc
#agg 1
Center (11574,22166); offset 3050,8694
Image size: 6480 x 5760;   To excise:
  trimvol -x 8334,14813 -y 19286,25045 -z 160,504 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg1.mrc
#agg 8
Center (9356,18207); offset 832,4735
Image size: 4640 x 5070;   To excise:
  trimvol -x 7036,11675 -y 15672,20741 -z 10,200 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg8.mrc
#agg 9
Center (10866,16152); offset 2342,2680
Image size: 2840 x 2520;   To excise:
  trimvol -x 9446,12285 -y 14892,17411 -z 10,170 /Volumes/Transfer/11-10073_SBEM2_bin1.mrc /Volumes/Transfer/extracted_aggregates/11-10073_SBEM2_bin1_agg9.mrc`

# convert to unsigned
by default the new subvolumes are stored as signed 16bit integers. We need to convert these these unsigned. This can be done using the 'newstack' command in IMOD. 

``` newstack -mode 6 /Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_agg9.mrc /Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_agg9_unsigned.mrc```

# Aggregate sub-volumes
## 1) key frames
Each of the aggregate sub-volumes are stored as separate mrc files. Each of these files needs to be opened, the dimensions recorded and the data extracted. The key frame identified in IMOD is extracted and added to a list and the pixel value histogram plotted. The mrc files are then converted to 16-bit arrays and the the key frame re-extracted and histogram re-plotted.

### define key frames of each aggregate sub-volume and open

In [ ]:
#define open and get info and return key frame
def open_mrc_keyf(file_name, key_frame):
    """open an mrc file and return the key frame as a np arrays"""
    mrc = mrcfile.mmap(file_name, mode='r')
    dim = '{}: there are {} slices each {} x {} pixels'.format(file_name, mrc.data.shape[0],mrc.data.shape[1],mrc.data.shape[2])
    print(dim)
    data = mrc.data[key_frame]
    mrc.close()
    return data

#define agg dict with agg name and key frame
agg_dict={'agg1':186, 'agg2':227, 'agg3':139, 'agg4':121, 'agg5':101, 'agg6':107, 'agg7':80, 'agg8':82, 'agg9':106}
#initialize list of key frames
agg_mrc_frames = []

for k, v in agg_dict.items():
    data = open_mrc_keyf(f'/Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_{k}_unsigned.mrc', v)
    #data_8bit = skimage.img_as_ubyte(data)
    #data_s = skimage.exposure.rescale_intensity(data, in_range='image', out_range=np.int16)
    #data_u = skimage.img_as_uint(data)
    print(data.dtype)
    agg_mrc_frames.append(data)
    #agg_dimensions.append(data[1])

print(f'there are {len(agg_mrc_frames)} key frames in agg_mrc_frames')
print(type(agg_mrc_frames))

In [ ]:
imgplot = plt.imshow(agg_mrc_frames[1])

### display each key frame

In [ ]:
pal = Greys[256]
im_names = agg_dict.keys()
interpixel_dist = 55
ims = agg_mrc_frames[3:9]

# Display images
plots = [
    bebi103.image.imshow(
        im, frame_height=300, title=name, interpixel_distance=0.0055, length_units="µm", cmap = pal
    )
    for name, im in zip(im_names, ims)
]

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

### make a histogram of pixel values for each key frame

In [ ]:
def plot_hist(im, title, logy=True):
    """Make plot of image histogram."""
    counts, vals = skimage.exposure.histogram(im)
    if logy:
        inds = counts > 0
        log_counts = np.log(counts[inds])
        return hv.Spikes(
            data=(vals[inds], log_counts),
            kdims=['pixel values'],
            vdims=['log₁₀ count'],
            label=title,
        ).opts(
            frame_height=100,
        )

    return hv.Spikes(
        data=(vals, counts),
        kdims=['pixel values'],
        vdims=['count'],
        label=title,
    ).opts(
        frame_height=100,
    )

im_names = agg_dict.keys()
ims = agg_mrc_frames

# Display histograms
plots = [plot_hist(im, name) for name, im in zip(im_names, ims)]
hv.Layout(
    plots
).opts(
    shared_axes=False,
).cols(
    3
)

#save layout
#export_png(plots, filename = '/Volumes/Transfer/extracted_aggregates/aggregate_key_frame_histograms.png')


# convert to npz

In [ ]:
#extract the data from each stack

def open_mrc_data(file_name):
    """open an mrc file and return the data as a set of np arrays"""
    mrc = mrcfile.mmap(file_name, mode='r')
    dim = '{}: there are {} slices each {} x {} pixels'.format(file_name, mrc.data.shape[0],mrc.data.shape[1],mrc.data.shape[2])
    print(dim)
    data = mrc.data
    mrc.close()
    return data

names = agg_dict.keys()
    
for i in names:
    stack_data = open_mrc_data(f'/Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_{i}_unsigned.mrc')
    X = np.expand_dims(stack_data, axis=3)
    y_shape = (stack_data.shape[0], stack_data.shape[1], stack_data.shape[2], 4)
    y = np.zeros(y_shape, dtype=np.uint16)
    np.savez(f'/Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_{i}_unsigned.npz', X=X, y=y)



In [ ]:
def open_mrc_data(file_name):
    """open an mrc file and return the data as a set of np arrays"""
    mrc = mrcfile.mmap(file_name, mode='r')
    dim = '{}: there are {} slices each {} x {} pixels'.format(file_name, mrc.data.shape[0],mrc.data.shape[1],mrc.data.shape[2])
    print(dim)
    data = mrc.data
    mrc.close
    return data

stack_data = open_mrc_data('/Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_agg4_unsigned.mrc')

X = np.expand_dims(stack_data, axis=3)
y_shape = (stack_data.shape[0], stack_data.shape[1], stack_data.shape[2], 4)
y = np.zeros(y_shape, dtype=np.uint16)
np.savez('/Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_agg4_unsigned.npz', X=X, y=y)

In [ ]:
test_agg = np.load('/Volumes/Haley3view/extracted_aggregates/11-10073_SBEM2_bin1_agg4_unsigned.npz')

#test_agg.files
test_agg['X'].shape

## 2) save subvolume stacks

In [ ]:
#define open and get info and return key frame
def open_mrc(file_name):
    """open an mrc file and return the key frame as a np arrays"""
    mrc = mrcfile.mmap(file_name, mode='r')
    dim = '{}: there are {} slices each {} x {} pixels'.format(file_name, mrc.data.shape[0],mrc.data.shape[1],mrc.data.shape[2])
    print(dim)
    data = mrc.data
    mrc.close
    return data

data = open_mrc()